In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

In [5]:
import psycopg2


In [24]:
def get_def_info(word, from_lang, to_lang, multisence_dict):
    multisence_dict[word] = []
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    print("ENG_DEFIN_EX", ex.text.strip())
                print(current_definition)
                multisence_dict[word].append(current_definition)
                current_definition = {}
                print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_def_info("mother", "english","russian",sence_dict)


https://dictionary.cambridge.org/dictionary/english-russian/mother_1
DEFINITION your female parent
RUS_DEFIN мать
ENG_DEFIN_EX a single mother
ENG_DEFIN_EX My mother and father are divorced.
{'definition': 'your female parent', 'local_word': 'мать', 'examples': ['a single mother', 'My mother and father are divorced.']}
===
DEFINITION the title of an important nun (= woman who lives in a female religious group)
RUS_DEFIN мать (церковное звание)
ENG_DEFIN_EX Mother Teresa
{'definition': 'the title of an important nun (= woman who lives in a female religious group)', 'local_word': 'мать (церковное звание)', 'examples': ['Mother Teresa']}
===
https://dictionary.cambridge.org/dictionary/english-russian/mother_2


In [8]:
sence_dict['mother']

{'lemma_definitions': [{'definition': 'your female parent',
   'local_word': 'мать',
   'examples': ['a single mother', 'My mother and father are divorced.']},
  {'definition': 'the title of an important nun (= woman who lives in a female religious group)',
   'local_word': 'мать (церковное звание)',
   'examples': ['Mother Teresa']}]}

In [20]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

In [23]:
def process_lemma_and_forms(word, dictionary):
    get_def_info(word, "english","russian",dictionary)
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return 0
    else:
        #print(a[0])
        word_id = req_res[0]
        print(req_res[2])
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    for wordform in cursor:
        print(wordform[0])
        get_def_info(wordform[0], "english","russian",dictionary,is_word_form = True, original_word = word )
        
big_diCKt = {}        
process_lemma_and_forms("dad",big_diCKt)
big_diCKt

https://dictionary.cambridge.org/dictionary/english-russian/dad_1
DEFINITION father
RUS_DEFIN папа
ENG_DEFIN_EX Can I go to the park, Dad?
{'definition': 'father', 'local_word': 'папа', 'examples': ['Can I go to the park, Dad?']}
===
https://dictionary.cambridge.org/dictionary/english-russian/dad_2
dad
dads
https://dictionary.cambridge.org/dictionary/english-russian/dads_1
https://dictionary.cambridge.org/dictionary/english-russian/dads_2
https://dictionary.cambridge.org/dictionary/english-russian/dads_3


{'dad': {'definitions': [{'definition': 'father',
    'local_word': 'папа',
    'examples': ['Can I go to the park, Dad?']}],
  'wordforms': {'dads': {'definitions': []}}}}

In [ ]:
def get_def_info(word, from_lang, to_lang, multisence_dict, is_word_form = False, original_word = None ):
    if is_word_form == False:
        multisence_dict[word] = {'definitions':[], 'wordforms':{}}
    else:
        multisence_dict[original_word]['wordforms'][word]={'definitions':[]}
        
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    print("ENG_DEFIN_EX", ex.text.strip())
                print(current_definition)
                if is_word_form == False:
                    multisence_dict[word]['definitions'].append(current_definition)
                else:
                    multisence_dict[original_word]['wordforms'][word]['definitions'].append(current_definition)
                current_definition = {}
                print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_def_info("mother", "english","russian",sence_dict)

